## Start

In [10]:
from nba_api.stats.endpoints import leaguegamelog, teamgamelog, boxscoreadvancedv3
from nba_api.stats.static import teams
import pandas as pd
import time
import warnings

warnings.filterwarnings("ignore")

In [11]:
season = "2019-20"

## Teams and League Info

In [12]:
teams_df = pd.DataFrame(teams.get_teams())
team_to_id = {team["full_name"]: team["id"] for team in teams_df.to_dict("records")}
team_to_id.update(
    {team["abbreviation"]: team["id"] for team in teams_df.to_dict("records")}
)
team_to_id.update({team["id"]: team["id"] for team in teams_df.to_dict("records")})
teams_df.head(5)

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Georgia,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966


In [13]:
# Get league game log for season
game_log = leaguegamelog.LeagueGameLog(season=season).get_data_frames()
assert len(game_log) == 1
games_df = game_log[0]

# filter games_df to only include home games (prevent duplicating for both teams)
games_df = games_df[~games_df.MATCHUP.str.contains("@")]
print(games_df.shape, len(games_df.GAME_ID.unique()))

# add opponent team_id and team_abbreviation columns
games_df.loc[:, "OPP_TEAM_ABBREVIATION"] = games_df["MATCHUP"].str.split(" vs. ").str[1]
games_df.loc[:, "OPP_TEAM_ID"] = games_df["OPP_TEAM_ABBREVIATION"].map(team_to_id)

# adjust WL to 1 for win and 0 for loss
games_df.loc[:, "WL"] = games_df["WL"].map({"W": 1, "L": 0, 1: 1, 0: 0})

# get rid of useless columns
print(games_df.columns)
games_df = games_df[
    [
        "GAME_ID",
        "TEAM_ID",
        "TEAM_ABBREVIATION",
        "OPP_TEAM_ID",
        "OPP_TEAM_ABBREVIATION",
        "PLUS_MINUS",
        "WL",
    ]
]

games_df.head(10)

(1059, 29) 1059
Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M',
       'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST',
       'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS', 'VIDEO_AVAILABLE',
       'OPP_TEAM_ABBREVIATION', 'OPP_TEAM_ID'],
      dtype='object')


,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,OPP_TEAM_ID,OPP_TEAM_ABBREVIATION,PLUS_MINUS,WL
0,0021900001,1610612761,TOR,1610612740,NOP,8,1
2,0021900002,1610612746,LAC,1610612747,LAL,10,1
4,0021900010,1610612759,SAS,1610612752,NYK,9,1
7,0021900012,1610612756,PHX,1610612758,SAC,29,1
8,0021900004,1610612754,IND,1610612765,DET,-9,0
9,0021900003,1610612766,CHA,1610612741,CHI,1,1
13,0021900008,1610612755,PHI,1610612738,BOS,14,1
14,0021900006,1610612751,BKN,1610612750,MIN,-1,0
15,0021900009,1610612742,DAL,1610612764,WAS,8,1
19,0021900005,1610612753,ORL,1610612739,CLE,9,1


## Team Game Stats

### Processing

In [14]:
def process_team_df(team_df, window_size=5, debug=False):
    team_df = team_df.iloc[::-1]
    team_df["TS"] = team_df["PTS"] / (2 * (team_df["FGA"] + 0.44 * team_df["FTA"]))
    team_df["EFG"] = (team_df["FGM"] + 0.5 * team_df["FG3M"]) / team_df["FGA"]
    team_df["WL"] = team_df["WL"].map({"W": 1, "L": 0, 1: 1, 0: 0})
    team_df["P_W_PCT"] = team_df["W_PCT"].shift(1)
    team_df["PAST_WL"] = (
        team_df["WL"].rolling(window=window_size, min_periods=1).mean().shift(1)
    )
    team_df["PAST_PTS"] = (
        team_df["PTS"].rolling(window=window_size, min_periods=1).mean().shift(1)
    )
    team_df["PAST_REB"] = (
        team_df["REB"].rolling(window=window_size, min_periods=1).mean().shift(1)
    )
    team_df["PAST_AST"] = (
        team_df["AST"].rolling(window=window_size, min_periods=1).mean().shift(1)
    )
    team_df["PAST_STL"] = (
        team_df["STL"].rolling(window=window_size, min_periods=1).mean().shift(1)
    )
    team_df["PAST_BLK"] = (
        team_df["BLK"].rolling(window=window_size, min_periods=1).mean().shift(1)
    )
    team_df["PAST_TOV"] = (
        team_df["TOV"].rolling(window=window_size, min_periods=1).mean().shift(1)
    )
    team_df["PAST_PF"] = (
        team_df["PF"].rolling(window=window_size, min_periods=1).mean().shift(1)
    )
    team_df["PAST_FG3_PCT"] = (
        team_df["FG3_PCT"].rolling(window=window_size, min_periods=1).mean().shift(1)
    )
    # team_df["PAST_TS"] = (
    #     team_df["TS"].rolling(window=window_size, min_periods=1).mean().shift(1)
    # )
    # team_df["PAST_EFG"] = (
    #     team_df["EFG"].rolling(window=window_size, min_periods=1).mean().shift(1)
    # )
    team_df["P_W-L"] = (team_df["W"] - team_df["L"]).shift(1)
    if debug:
        debug_df = team_df[
            [
                "Game_ID",
                "W_PCT",
                "P_W_PCT",
                "WL",
                "PAST_WL",
                "PTS",
                "PAST_PTS",
                "W",
                "L",
                "P_W-L",
            ]
        ]
        print(debug_df)
    team_df = team_df[
        [
            "Game_ID",
            "GAME_DATE",
            "P_W_PCT",
            "PAST_WL",
            "PAST_PTS",
            "P_W-L",
            "PAST_REB",
            "PAST_AST",
            "PAST_STL",
            "PAST_BLK",
            "PAST_TOV",
            "PAST_PF",
            "PAST_FG3_PCT",
            # "PAST_TS",
            # "PAST_EFG",
        ]
    ]
    return team_df

In [15]:
# # Testing process_team_df function
# test_df = teamgamelog.TeamGameLog(
#     team_id=team_to_id["Los Angeles Lakers"], season=season
# ).get_data_frames()[0]
# print(test_df.columns)
# test_df = process_team_df(test_df, debug=True)
# test_df.head(10)

### Combining

In [16]:
team_dfs = {}
for i, team in enumerate(teams_df["id"]):
    print(f"Getting game log for {teams_df['full_name'][i]}")
    team_log = teamgamelog.TeamGameLog(team_id=team, season=season).get_data_frames()
    assert len(team_log) == 1
    team_dfs[teams_df["abbreviation"][i]] = process_team_df(team_log[0])
    time.sleep(0.1)

Getting game log for Atlanta Hawks
Getting game log for Boston Celtics
Getting game log for Cleveland Cavaliers
Getting game log for New Orleans Pelicans
Getting game log for Chicago Bulls
Getting game log for Dallas Mavericks
Getting game log for Denver Nuggets
Getting game log for Golden State Warriors
Getting game log for Houston Rockets
Getting game log for Los Angeles Clippers
Getting game log for Los Angeles Lakers
Getting game log for Miami Heat
Getting game log for Milwaukee Bucks
Getting game log for Minnesota Timberwolves
Getting game log for Brooklyn Nets
Getting game log for New York Knicks
Getting game log for Orlando Magic
Getting game log for Indiana Pacers
Getting game log for Philadelphia 76ers
Getting game log for Phoenix Suns
Getting game log for Portland Trail Blazers
Getting game log for Sacramento Kings
Getting game log for San Antonio Spurs
Getting game log for Oklahoma City Thunder
Getting game log for Toronto Raptors
Getting game log for Utah Jazz
Getting game 

In [17]:
comp_team_dfs = pd.concat(
    [df.assign(TEAM_ABBREVIATION=abbr) for abbr, df in team_dfs.items()]
)

In [18]:
full_df = games_df.merge(
    comp_team_dfs,
    left_on=["GAME_ID", "TEAM_ABBREVIATION"],
    right_on=["Game_ID", "TEAM_ABBREVIATION"],
    how="left",
)
full_df = full_df.merge(
    comp_team_dfs,
    left_on=["GAME_ID", "OPP_TEAM_ABBREVIATION"],
    right_on=["Game_ID", "TEAM_ABBREVIATION"],
    how="left",
    suffixes=("", "_OPP"),
)
full_df = full_df.drop(columns=["Game_ID", "TEAM_ABBREVIATION_OPP", "Game_ID_OPP"])

In [19]:
full_df.head(5)

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,OPP_TEAM_ID,OPP_TEAM_ABBREVIATION,PLUS_MINUS,WL,GAME_DATE,P_W_PCT,PAST_WL,...,PAST_WL_OPP,PAST_PTS_OPP,P_W-L_OPP,PAST_REB_OPP,PAST_AST_OPP,PAST_STL_OPP,PAST_BLK_OPP,PAST_TOV_OPP,PAST_PF_OPP,PAST_FG3_PCT_OPP
0,0021900001,1610612761,TOR,1610612740,NOP,8,1,"OCT 22, 2019",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0021900002,1610612746,LAC,1610612747,LAL,10,1,"OCT 22, 2019",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0021900010,1610612759,SAS,1610612752,NYK,9,1,"OCT 23, 2019",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0021900012,1610612756,PHX,1610612758,SAC,29,1,"OCT 23, 2019",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0021900004,1610612754,IND,1610612765,DET,-9,0,"OCT 23, 2019",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Advanced Stats

### Combining

In [ ]:
# Failed attempt at grabbing all offensive and defensive ratings + other advanced stats for each game (takes too long)
# boxscore_dfs = []
# tot = len(full_df)
# for i, game_row in full_df.iterrows():
#     print("Getting boxscore for", game_row["GAME_ID"], f"{i+1}/{tot}")
#     print(game_row["GAME_ID"])
#     boxscore_df = boxscoreadvancedv3.BoxScoreAdvancedV3(
#         game_id=game_row["GAME_ID"]
#     ).get_data_frames()[1]
#     boxscore_dfs.append(boxscore_df)
#     time.sleep(0.1)

In [22]:
# asdflasdkjf = boxscoreadvancedv3.BoxScoreAdvancedV3(
#     game_id="0021900002"
# ).get_data_frames()
# print(asdflasdkjf[1].columns)
# asdflasdkjf[1].head(10)

### ELO

In [30]:
elo_k = 20
team_elo = {team: 1500 for team in teams_df["abbreviation"]}

# create 2 new columns for elo ratings
full_df["TEAM_ELO"] = 1500
full_df["OPP_TEAM_ELO"] = 1500


def prob_win(a, b):
    return 1 / (1 + 10 ** ((team_elo[b] - team_elo[a]) / 400))


# def update_amount(winner, loser):
#     return elo_k * (1 - prob_win(winner, loser))

In [33]:
# loop through full_df and update elo ratings
for i, game_row in full_df.iterrows():
    full_df.loc[i, "TEAM_ELO"] = team_elo[game_row["TEAM_ABBREVIATION"]]
    full_df.loc[i, "OPP_TEAM_ELO"] = team_elo[game_row["OPP_TEAM_ABBREVIATION"]]
    winner = (
        game_row["TEAM_ABBREVIATION"]
        if game_row["WL"] == 1
        else game_row["OPP_TEAM_ABBREVIATION"]
    )
    loser = (
        game_row["TEAM_ABBREVIATION"]
        if game_row["WL"] == 0
        else game_row["OPP_TEAM_ABBREVIATION"]
    )
    d_update = elo_k * (1 - prob_win(winner, loser))
    team_elo[winner] += d_update
    team_elo[loser] -= d_update

## Final Processing

In [35]:
full_df.columns

Index(['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'OPP_TEAM_ID',
       'OPP_TEAM_ABBREVIATION', 'PLUS_MINUS', 'WL', 'GAME_DATE', 'P_W_PCT',
       'PAST_WL', 'PAST_PTS', 'P_W-L', 'PAST_REB', 'PAST_AST', 'PAST_STL',
       'PAST_BLK', 'PAST_TOV', 'PAST_PF', 'PAST_FG3_PCT', 'GAME_DATE_OPP',
       'P_W_PCT_OPP', 'PAST_WL_OPP', 'PAST_PTS_OPP', 'P_W-L_OPP',
       'PAST_REB_OPP', 'PAST_AST_OPP', 'PAST_STL_OPP', 'PAST_BLK_OPP',
       'PAST_TOV_OPP', 'PAST_PF_OPP', 'PAST_FG3_PCT_OPP', 'TEAM_ELO',
       'OPP_TEAM_ELO'],
      dtype='object')

In [36]:
# count rows with na values
print(full_df.shape, full_df.isna().sum().sum())
df = full_df[
    [
        "GAME_ID",
        "TEAM_ABBREVIATION",
        "OPP_TEAM_ABBREVIATION",
        "PLUS_MINUS",
        "WL",
        "GAME_DATE",
        "P_W_PCT",
        "PAST_WL",
        "PAST_PTS",
        "P_W-L",
        "PAST_REB",
        "PAST_AST",
        "PAST_STL",
        "PAST_BLK",
        "PAST_TOV",
        "PAST_PF",
        "PAST_FG3_PCT",
        "P_W_PCT_OPP",
        "PAST_WL_OPP",
        "PAST_PTS_OPP",
        "P_W-L_OPP",
        "PAST_REB_OPP",
        "PAST_AST_OPP",
        "PAST_STL_OPP",
        "PAST_BLK_OPP",
        "PAST_TOV_OPP",
        "PAST_PF_OPP",
        "PAST_FG3_PCT_OPP",
        "TEAM_ELO",
        "OPP_TEAM_ELO",
    ]
]
df.dropna(inplace=True)
df.head(10)

(1059, 33) 330


,GAME_ID,TEAM_ABBREVIATION,OPP_TEAM_ABBREVIATION,PLUS_MINUS,WL,GAME_DATE,P_W_PCT,PAST_WL,PAST_PTS,P_W-L,...,P_W-L_OPP,PAST_REB_OPP,PAST_AST_OPP,PAST_STL_OPP,PAST_BLK_OPP,PAST_TOV_OPP,PAST_PF_OPP,PAST_FG3_PCT_OPP,TEAM_ELO,OPP_TEAM_ELO
16,0021900022,OKC,WAS,-12,0,"OCT 25, 2019",0.0,0.0,95.0,-1.0,...,-1.0,47.0,26.0,6.0,3.0,17.0,32.0,0.268,1583.374043,1390.394879
17,0021900024,SAC,POR,-10,0,"OCT 25, 2019",0.0,0.0,95.0,-1.0,...,-1.0,48.0,13.0,9.0,4.0,20.0,25.0,0.250,1476.484365,1516.840277
18,0021900017,BOS,TOR,6,1,"OCT 25, 2019",0.0,0.0,93.0,-1.0,...,1.0,57.0,23.0,7.0,3.0,16.0,24.0,0.350,1584.933942,1679.707413
19,0021900019,BKN,NYK,4,1,"OCT 25, 2019",0.0,0.0,126.0,-1.0,...,-1.0,39.0,24.0,16.0,1.0,14.0,32.0,0.333,1498.791727,1398.936568
20,0021900018,CHA,MIN,-22,0,"OCT 25, 2019",1.0,1.0,126.0,1.0,...,1.0,52.0,23.0,9.0,4.0,13.0,22.0,0.302,1421.582476,1367.954009
21,0021900025,LAL,UTA,9,1,"OCT 25, 2019",0.0,0.0,102.0,-1.0,...,1.0,49.0,15.0,4.0,5.0,11.0,31.0,0.250,1620.279133,1563.346541
22,0021900021,NOP,DAL,-7,0,"OCT 25, 2019",0.0,0.0,122.0,-1.0,...,1.0,46.0,17.0,6.0,5.0,19.0,22.0,0.306,1463.165922,1527.668967
23,0021900020,MEM,CHI,-8,0,"OCT 25, 2019",0.0,0.0,101.0,-1.0,...,-1.0,49.0,25.0,11.0,4.0,10.0,20.0,0.300,1481.934747,1369.559127
24,0021900023,DEN,PHX,1,1,"OCT 25, 2019",1.0,1.0,108.0,1.0,...,1.0,44.0,31.0,13.0,9.0,14.0,25.0,0.333,1567.656086,1540.588460
25,0021900035,PHX,LAC,8,1,"OCT 26, 2019",0.5,0.5,115.5,0.0,...,2.0,42.0,28.5,7.5,5.0,15.5,24.5,0.459,1531.365956,1634.786908


In [38]:
df.to_csv(f"nba_{season}_data_updated.csv", index=False)